In [3]:
import sys, os
import json
from typing import Any, Union, Dict, List
import humpack
from humpack import tdict, tlist, tset

In [2]:
humpack.__version__

'0.1'

In [3]:
P = (type(None), str, float, int, bool)
P

(NoneType, str, float, int, bool)

In [4]:
Union[P]

typing.Union[NoneType, str, float, int, bool]

In [5]:
humpack.PRIMITIVE

typing.Union[NoneType, str, int, float, bool]

In [6]:
Union[P]

typing.Union[NoneType, str, float, int, bool]